In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iit-patna-movie-reviews-hindi/iitp-movie-reviews/readme.md
/kaggle/input/iit-patna-movie-reviews-hindi/iitp-movie-reviews/hi/hi-train.csv
/kaggle/input/iit-patna-movie-reviews-hindi/iitp-movie-reviews/hi/hi-valid.csv
/kaggle/input/iit-patna-movie-reviews-hindi/iitp-movie-reviews/hi/hi-test.csv


In [63]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.models import Model
import string
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [62]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize, pos_tag
import os, re, csv, math, codecs
from sklearn import model_selection
from sklearn import metrics
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

In [2]:
df_train = pd.read_csv('/kaggle/input/iit-patna-movie-reviews-hindi/iitp-movie-reviews/hi/hi-train.csv')
df_valid = pd.read_csv('/kaggle/input/iit-patna-movie-reviews-hindi/iitp-movie-reviews/hi/hi-valid.csv')
df_test = pd.read_csv('/kaggle/input/iit-patna-movie-reviews-hindi/iitp-movie-reviews/hi/hi-test.csv')
df_train.columns = ['Sentiment', 'Text']
df_valid.columns = ['Sentiment', 'Text']
df_test.columns = ['Sentiment', 'Text']

df_train.head()

,Sentiment,Text
0,positive,’उड़ान’ से विक्रमादित्य\nमोटवाने\nने अच्छे सिन...
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...
4,positive,इन फुकरों से आप पहले भी मिल चुके हैं और आपने इ...


In [3]:
df_test.head()

,Sentiment,Text
0,negative,"निर्माता :\nफरहान अख्तर, रितेश सिधवानी\n\nनिर्..."
1,positive,असमर्पित रिश्ते में यकीन रखने वाले दोनों मौज-म...
2,neutral,"बेवकूफियां को हबीब फैज़ल ने लिखा है, जिनका नाम..."
3,neutral,तब वह मोटी थी और जेरी ने उस पर ध्यान नहीं दिय...
4,positive,'तेवर' में निर्देशक अमित रवींद्रनाथ शर्मा ने अ...


In [4]:
# Cleaning the data
df_train['Text'] = df_train['Text'].str.replace('[,’।"\n\']', '', regex=True)
df_valid['Text'] = df_valid['Text'].str.replace('[,’।"\n\']', '', regex=True)
df_test['Text'] = df_test['Text'].str.replace('[,’।"\n\']', '', regex=True)

In [8]:
from googletrans import Translator

In [9]:
# Initialize the translator
translator = Translator()

In [10]:
 df_train['Text_en'] = df_train['Text'].apply(lambda text: translator.translate(text, src='hi', dest='en').text)

In [13]:
df_train.head(10)

,Sentiment,Text,Text_en
0,positive,उड़ान से विक्रमादित्यमोटवानेने अच्छे सिनेमा की...,उड़ान से विक्रमादित्यमोटवानेने अच्छे सिनेमा की...
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...,"In the song scenes in the film, Anushka had to..."
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...,"Before being hanged, she takes special permiss..."
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...,Raj Kumar Hirani has shown the magic of his di...
4,positive,इन फुकरों से आप पहले भी मिल चुके हैं और आपने इ...,इन फुकरों से आप पहले भी मिल चुके हैं और आपने इ...
5,negative,अतार्किक दृश्यों और भिड़ंत से फिल्म कमजोर हो ग...,The film is weakened by illogical scenes and c...
6,positive,चारों दिशाओं को थर्रा देने का उनमें दंभ है,They have the arrogance to make all four direc...
7,positive,शाहिद और जेनेलिया के बीच गजब की केमिस्ट्री है,There is amazing chemistry between Shahid and ...
8,negative,निखिल आडवाणी उन निर्देशकों में से हैं जो अच्छी...,निखिल आडवाणी उन निर्देशकों में से हैं जो अच्छी...
9,neutral,मूल तेलुगू में किक देख चुके दर्शक बता सकेगे कि...,Viewers who have seen Kick in the original Tel...


In [22]:
df_train.shape

(2479, 3)

In [23]:
df_train = df_train[df_train['Text'] != df_train['Text_en']]

In [24]:
df_train.shape

(1830, 3)

In [26]:
df_train.head()

,Sentiment,Text,Text_en
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...,"In the song scenes in the film, Anushka had to..."
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...,"Before being hanged, she takes special permiss..."
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...,Raj Kumar Hirani has shown the magic of his di...
5,negative,अतार्किक दृश्यों और भिड़ंत से फिल्म कमजोर हो ग...,The film is weakened by illogical scenes and c...
6,positive,चारों दिशाओं को थर्रा देने का उनमें दंभ है,They have the arrogance to make all four direc...


In [40]:

# Importing LabelEncoder from Sklearn 
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label_fit = le.fit(df_train['Sentiment'])
 
# printing label
label = label_fit.transform(df_train['Sentiment'])


In [41]:
df_train['label'] = label

In [42]:
df_train.head()

,Sentiment,Text,Text_en,label
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...,"In the song scenes in the film, Anushka had to...",1
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...,"Before being hanged, she takes special permiss...",1
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...,Raj Kumar Hirani has shown the magic of his di...,2
5,negative,अतार्किक दृश्यों और भिड़ंत से फिल्म कमजोर हो ग...,The film is weakened by illogical scenes and c...,0
6,positive,चारों दिशाओं को थर्रा देने का उनमें दंभ है,They have the arrogance to make all four direc...,2


In [43]:
df_valid['label'] = label_fit.transform(df_valid['Sentiment'])

In [44]:
df_valid.head()

,Sentiment,Text,label
0,neutral,निर्माता :आदित्य चोपड़ानिर्देशक :हबीब फैजलसंगी...,1
1,positive,इस कसौटी पर करण काफी हद तक खरे भी उतरे हैं,2
2,positive,छोटे शहर की गालियां छतें और विहंगम दृश्य मोहक ...,2
3,positive,मनीष और जिशान का काम भी लोगों को याद रहेगा,2
4,positive,बैनर :एसएलबी फिल्म्स हरि ॐ एंटरटेनमेंट कंपनी य...,2


In [45]:
df_test['label'] = label_fit.transform(df_test['Sentiment'])

In [51]:
import spacy
import re
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

In [52]:
#hindi
def remove_punct_hi(text):
    new_words = " "
    for word in text:
        if word not in STOP_WORDS_HI:
            w = re.sub(r'[.#-:?!\_|""'',-...A-Za-z0-9]','',word) #remove everything except words and space#how 
            new_words = new_words + w
    return re.sub(r"\s+", " ", new_words, flags=re.UNICODE)


In [53]:
df_train['Text'] = df_train['Text'].apply(lambda x: remove_punct_hi(x))

In [59]:
df_train['Sentiment'].value_counts()

Sentiment
positive    786
negative    539
neutral     505
Name: count, dtype: int64

In [71]:
X_train = list(df_train['Text'])

In [73]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [74]:
words_to_index = tokenizer.word_index

In [76]:
len(words_to_index)

8031

In [77]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding = "ISO-8859-1") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:])



    return word_to_vec_map

In [83]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz

--2023-12-26 19:06:45--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.103, 18.244.202.25, 18.244.202.73, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1118942272 (1.0G) [binary/octet-stream]
Saving to: 'cc.hi.300.vec.gz'

cc.hi.300.vec.gz    100%[===================>]   1.04G  45.3MB/s    in 24s     

2023-12-26 19:07:09 (43.7 MB/s) - 'cc.hi.300.vec.gz' saved [1118942272/1118942272]



In [87]:
!gzip -d /kaggle/working/cc.hi.300.vec.gz